In [3]:
!pip install git+https://github.com/PrithivirajDamodaran/Parrot.git >> None
!pip install nltk >> None
!pip install rouge >> None
!pip install datasets >> None
!pip install pyarrow==6.0.1 >> None
!pip install datasets==2.3.2 >> None
!pip install sentencepiece >> None
!pip install transformers >> None
!pip install pytorch-lightning >> None
!pip install rouge_score >> None

In [20]:
# os
import warnings
import time
import tqdm
import random
import string
import re
import csv
from pathlib import Path
from tqdm import tqdm
import argparse
import glob
import os
import json
import logging
from itertools import chain
from string import punctuation
import sentencepiece
from random import shuffle

# модели
import torch
from parrot import Parrot
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import MarianMTModel, MarianTokenizer
from concurrent.futures import ThreadPoolExecutor, as_completed
from transformers import AutoModelForSequenceClassification, BertTokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# метрики
import nltk
from nltk.translate.bleu_score import sentence_bleu
#from datasets import load_metric
from rouge_score import rouge_scorer
from nltk.corpus import wordnet
#from nltk.translate.ter_score import ter_score

warnings.filterwarnings("ignore")
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context


nltk.download('punkt', )
nltk.download('wordnet', )
nltk.download('all', )

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cotangentofzero/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/cotangentofzero/nltk_data...
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/cotangentofzero/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/cotangentofzero/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/cotangentofzero/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/cotangentofzero/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_

True

In [ ]:
def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [7]:
full_text = """
“Wings of Fire: An Autobiography of Abdul Kalam”  is written by Dr. APJ Abdul Kalam and Arun Tiwari. This book is an autobiography of Dr.
APJ Abdul Kalam, former president of India. This story sheds light on the journey of a young boy from Rameswaram to become a renowned scientist.
It reflects how simple living, dedication, strong will and hard work led to success. It also shows how cultural unity impacts the lives of individuals.
"""

## 2. Перефразирование текста по исходному запросу (T5) (+)

In [ ]:
def create_paraphrase_T5(full_text, max_length, num_return_sequences, early_stopping, type_model):

    model_1 = "t5-base"
    model_2 = "t5-large"

    model = T5ForConditionalGeneration.from_pretrained(model_2)
    tokenizer = T5Tokenizer.from_pretrained(model_2)

    input_text = full_text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    output_ids = model.generate(input_ids,
                                max_length=max_length,
                                num_return_sequences=num_return_sequences,
                                early_stopping=early_stopping)
    t5_output_text = tokenizer.decode(output_ids[0],
                                      skip_special_tokens=True)

    # Вычисление BLEU
    bleu_score = sentence_bleu([full_text.split()], t5_output_text.split())
    print(f"BLEU Score: {bleu_score:.2f}")

    return t5_output_text

In [ ]:
%%time

# max_length, num_return_sequences, early_stopping - добавляем на них крутилки
T5_paraphrased = create_paraphrase_T5(full_text, max_length=80, num_return_sequences=1, early_stopping=True)

print("Original text: \n")
print(full_text)
print("\nT5 paraphrased text: \n")
print(T5_paraphrased)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


BLEU Score: 0.06
Original text: 


“Wings of Fire: An Autobiography of Abdul Kalam”  is written by Dr. APJ Abdul Kalam and Arun Tiwari. This book is an autobiography of Dr.
APJ Abdul Kalam, former president of India. This story sheds light on the journey of a young boy from Rameswaram to become a renowned scientist.
It reflects how simple living, dedication, strong will and hard work led to success. It also shows how cultural unity impacts the lives of individuals.


T5 paraphrased text: 

. “Wings of Fire” is written by Arun Tiwari. This book is written by Arun Tiwari. This book is written by Arun Tiwari. This book “Wings of Fire” is written by Arun Tiwari. This book is written by Arun Tiwari. “Wings Fire
CPU times: user 23.9 s, sys: 3.86 s, total: 27.8 s
Wall time: 55 s


## 2. Перефразмирование текста по исходному запросу (Bart) (+)

In [ ]:
def create_paraphrase_bart(full_text, max_length, num_return_sequences, early_stopping, type_model="facebook/bart-large"):

    model = BartForConditionalGeneration.from_pretrained(type_model)
    tokenizer = BartTokenizer.from_pretrained(type_model)

    input_text = full_text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    output_ids = model.generate(input_ids,
                                max_length=max_length,
                                num_return_sequences=num_return_sequences,
                                early_stopping=early_stopping)
    output_text_bart = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Оценка качества перефразирования
    bleu_score = sentence_bleu([full_text.split()], output_text_bart.split())
    print(f"BLEU Score: {bleu_score:.2f}")

    print(f"Перефразированный текст: {output_text_bart}")

    return output_text_bart

In [ ]:
%%time

# max_length, num_return_sequences, early_stopping - добавляем на них крутилки
bart_paraphrased = create_paraphrase_bart(full_text, max_length=100, num_return_sequences=1, early_stopping=True)

print("Original text: \n")
print(full_text)
print("\nBart paraphrased text: \n")
print(bart_paraphrased)

BLEU Score: 0.67
Перефразированный текст: “Wings of Fire: An Autobiography of Abdul Kalam”  is written by Dr. APJ AbdulKalam and Arun Tiwari. This book is an autobiography of Dr. A.K.A.APJ Abdul KalAM, former president of India. This story sheds light on the journey of a young boy from Rameswaram to become a renowned scientist.It reflects how simple living, dedication, strong will and hard work led to
Original text: 


“Wings of Fire: An Autobiography of Abdul Kalam”  is written by Dr. APJ Abdul Kalam and Arun Tiwari. This book is an autobiography of Dr.
APJ Abdul Kalam, former president of India. This story sheds light on the journey of a young boy from Rameswaram to become a renowned scientist.
It reflects how simple living, dedication, strong will and hard work led to success. It also shows how cultural unity impacts the lives of individuals.


Bart paraphrased text: 

“Wings of Fire: An Autobiography of Abdul Kalam”  is written by Dr. APJ AbdulKalam and Arun Tiwari. This book is an

## 2. Перефразирование текста по исходному запросу (T5 + Parrot) (+)

In [ ]:
def create_paraphrase(full_text, use_gpu, diversity_ranker, do_diverse, max_return_phrases, max_length, adequacy_threshold, fluency_threshold):
    # Использование модели T5 для перефразирования
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    tokenizer = T5Tokenizer.from_pretrained("t5-base")

    input_text = full_text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    output_ids = model.generate(input_ids, max_length=100, num_return_sequences=1, early_stopping=True)
    t5_output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Использование библиотеки Parrot для дополнительного перефразирования
    parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=True)

    phrases = [x.strip() for x in full_text.split('.')]
    output_phrases = []

    for phrase in phrases:
        if len(phrase) > 1:
            para_phrases = parrot.augment(input_phrase=phrase,
                                          use_gpu=False,
                                          diversity_ranker="levenshtein",
                                          do_diverse=False,
                                          max_return_phrases=10,
                                          max_length=32,
                                          adequacy_threshold=0.99,
                                          fluency_threshold=0.90)

            try:
                for para_phrase in para_phrases:
                    (x, y) = para_phrase
                    x = x[0].upper() + x[1:] # capitalize
                    output_phrases.append(x)
                    break # just get the first phrase
            except:
                print("Exception occurred with this one.")

    parrot_output_text = ".".join(output_phrases)

    # Оценка качества перефразирования
    bleu_score = sentence_bleu([full_text.split()], parrot_output_text.split())
    print(f"BLEU Score: {bleu_score:.2f}")

    return t5_output_text, parrot_output_text

In [ ]:
%%time

t5_paraphrased, parrot_paraphrased = create_paraphrase(full_text,

                                                       max_return_phrases=10,
                                                       max_length=32,
                                                       adequacy_threshold=0.99,
                                                       fluency_threshold=0.90)

print("\nT5 paraphrased text:")
print(t5_paraphrased)
print("\nParrot paraphrased text:")
print(parrot_paraphrased)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Exception occurred with this one.
Exception occurred with this one.
Exception occurred with this one.
Exception occurred with this one.
Exception occurred with this one.
BLEU Score: 0.00

T5 paraphrased text:
“Wings of Fire” is a book by Dr. APJ Abdul Kalam and Arun Tiwari. is a collaboration between Dr. APJ Abdul Kalam and Arun Tiwari. Abdul Kalam..

Parrot paraphrased text:
APJ Abdul Kalam and Arun Tiwari.APJ Abdul Kalam, former president of India
CPU times: user 1min 17s, sys: 3.22 s, total: 1min 20s
Wall time: 1min 47s


## 2. Перефразирование текста по исходному запросу (Language) (+)

In [ ]:
# Функция для перефразирования текста
def paraphrase(text):

    # Загрузка моделей машинного перевода
    src_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
    src_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")

    tgt_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
    tgt_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

    # Перевод текста с английского на русский
    input_ids = src_tokenizer.encode(text, return_tensors="pt")
    output_ids = src_model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
    paraphrased_text_ru = src_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Перевод обратно с русского на английский
    input_ids = tgt_tokenizer.encode(paraphrased_text_ru, return_tensors="pt")
    output_ids = tgt_model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
    paraphrased_text = tgt_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return paraphrased_text

In [ ]:
# Пример использования
original_text = "The quick brown fox jumps over the lazy dog."
paraphrased_text = paraphrase(original_text)
print("Original text:", original_text)
print("Paraphrased text:", paraphrased_text)

## 2. Перефразирование текста по исходному запросу (Русский язык) (+)

In [ ]:
MODEL_NAME = 'cointegrated/rut5-base-paraphraser'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def paraphrase(text, beams=5, grams=4, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)

print(paraphrase('Каждый охотник желает знать, где сидит фазан.'))

## 2. Перефразирование текста по исходному запросу (Pegasus) (+)

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from itertools import product
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from tqdm import tqdm

model_1 = "tuner007/pegasus_paraphrase"
model_2 = "google/pegasus-xsum"

# Загрузка модели и токенайзера
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

# Функция перефразирования текста
def rephrase_text(text, max_length, num_beams, early_stopping, num_return_sequences):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    output_ids = model.generate(
        input_ids,
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=early_stopping,
        num_return_sequences=num_return_sequences
    )[0]

    return tokenizer.decode(output_ids, skip_special_tokens=True)

# Определение сетки перебора
param_grid = {
    'max_length': [50, 100, 150, 200],
    'num_beams': [2, 4, 8, 16],
    'early_stopping': [True, False],
    'num_return_sequences': [1, 3, 5]
}

# Создание всех комбинаций гиперпараметров
all_param_combinations = list(product(*param_grid.values()))

# Оригинальный текст для перефразирования
original_text = "Artificial intelligence has made significant advancements in recent years, with applications in various fields like healthcare, finance, and transportation."

# Оценка и сравнение моделей
best_score = 0
best_params = None
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

for params in tqdm(all_param_combinations, desc="Evaluating parameter combinations"):
    max_length, num_beams, early_stopping, num_return_sequences = params

    # Пропускаем несовместимые комбинации
    if num_return_sequences > num_beams:
        continue

    rephrased_text = rephrase_text(original_text, max_length, num_beams, early_stopping, num_return_sequences)

    # Оценка качества перефразирования
    rouge_scores = scorer.score(original_text, rephrased_text)
    bleu_score = sentence_bleu([original_text.split()], rephrased_text.split())

    # Комбинирование метрик
    total_score = 0.5 * (rouge_scores['rouge1'].fmeasure + rouge_scores['rouge2'].fmeasure + rouge_scores['rougeL'].fmeasure) + 0.5 * bleu_score

    if total_score > best_score:
        best_score = total_score
        best_params = {
            'max_length': max_length,
            'num_beams': num_beams,
            'early_stopping': early_stopping,
            'num_return_sequences': num_return_sequences
        }

print(f"Best parameters: {best_params}")
print(f"Best score: {best_score:.4f}")

## 3. Камуфлирование текста

In [22]:
def modify_syntax(text):
    modified_text = text

    # Генерация случайных порогов вероятности
    add_spaces_around_punctuation = random.uniform(0.74, 0.9)
    remove_extra_punctuation = random.uniform(0.6, 0.8)
    replace_question_exclamation = random.uniform(0.5, 0.74)
    normalize_ellipsis = random.uniform(0.7, 0.9)
    add_spaces_around_non_alphanumeric = random.uniform(0.75, 1.0)

    # Добавление пробелов вокруг знаков препинания
    if random.random() < add_spaces_around_punctuation:
        modified_text = re.sub(r'([,.\?!])', r' \1 ', modified_text)
        if random.random() < 0.5:
            modified_text = re.sub(r' +', ' ', modified_text)

    # Удаление лишних знаков препинания
    if random.random() < remove_extra_punctuation:
        modified_text = re.sub(r'[,\.]+(?=[\)\]"])', '', modified_text)
        if random.random() < 0.3:
            modified_text = re.sub(r'[,\.]+', random.choice([',', '.', '']), modified_text)

    # Замена вопросительного знака с восклицательным
    if random.random() < replace_question_exclamation:
        modified_text = re.sub(r'\?!', random.choice(['?', '!']), modified_text)

    # Нормализация использования многоточий
    if random.random() < normalize_ellipsis:
        modified_text = re.sub(r'\.{2,}', '...', modified_text)
        if random.random() < 0.4:
            modified_text = re.sub(r'\.{3}', random.choice(['...', '...']), modified_text)

    # Добавление пробелов вокруг непечатных символов
    if random.random() < add_spaces_around_non_alphanumeric:
        modified_text = re.sub(r'([^a-zA-Z0-9])', r' \1 ', modified_text)
        if random.random() < 0.6:
            modified_text = re.sub(r' +', ' ', modified_text)

    # Удаление лишних пробелов
    modified_text = re.sub(r'\s+', ' ', modified_text)

    return modified_text

In [23]:
modify_syntax(full_text)

' “ Wings of Fire : An Autobiography of Abdul Kalam ” is written by Dr . APJ Abdul Kalam and Arun Tiwari . This book is an autobiography of Dr . APJ Abdul Kalam , former president of India . This story sheds light on the journey of a young boy from Rameswaram to become a renowned scientist . It reflects how simple living , dedication , strong will and hard work led to success . It also shows how cultural unity impacts the lives of individuals . '

In [9]:
# Изменениие длины предложений
import re
def modify_sentence_length(text):
    sentences = re.split(r'[.!?]+', text)

    modified_sentences = []
    for sentence in sentences:
        if len(sentence.split()) > 20:
            short_sentences = re.split(r'[,;]+', sentence)
            modified_sentences.extend(short_sentences)
        else:
            if len(modified_sentences) > 0 and len(modified_sentences[-1].split()) < 10:
                modified_sentences[-1] += ' ' + sentence.strip()
            else:
                modified_sentences.append(sentence.strip())

    modified_text = '. '.join(modified_sentences) + '.'

    return modified_text

original_text = "Wings of Fire: An Autobiography of Abdul Kalam is written by Dr. APJ Abdul Kalam and Arun Tiwari. This book is an autobiography of Dr. APJ Abdul Kalam, former president of India. This story sheds light on the journey of a young boy from Rameswaram to become a renowned scientist. It reflects how simple living, dedication, strong will and hard work led to success. It also shows how cultural unity impacts the lives of individuals."

modified_text = modify_sentence_length(original_text)

# print("Оригинальный текст:")
# print(original_text)
print("\nМодифицированный текст:")
print(modified_text)


Модифицированный текст:
Wings of Fire: An Autobiography of Abdul Kalam is written by Dr. APJ Abdul Kalam and Arun Tiwari This book is an autobiography of Dr. APJ Abdul Kalam, former president of India This story sheds light on the journey of a young boy from Rameswaram to become a renowned scientist. It reflects how simple living, dedication, strong will and hard work led to success. It also shows how cultural unity impacts the lives of individuals. .


In [10]:
# Перестановка слов
def modify_word_order(text):
    sentences = re.split(r'[.!?]+', text)

    modified_sentences = []
    for sentence in sentences:
        words = sentence.split()

        if len(words) > 2:
            middle_words = words[1:-1]
            shuffle(middle_words)

            modified_sentence = ' '.join([words[0]] + middle_words + [words[-1]])
        else:
            modified_sentence = ' '.join(words)

        modified_sentences.append(modified_sentence)

    modified_text = '. '.join(modified_sentences) + '.'

    return modified_text

original_text = "Wings of Fire: An Autobiography of Abdul Kalam is written by Dr. APJ Abdul Kalam and Arun Tiwari. This book is an autobiography of Dr. APJ Abdul Kalam, former president of India. This story sheds light on the journey of a young boy from Rameswaram to become a renowned scientist. It reflects how simple living, dedication, strong will and hard work led to success. It also shows how cultural unity impacts the lives of individuals."

modified_text = modify_word_order(original_text)

# print("Оригинальный текст:")
# print(original_text)
print("\nМодифицированный текст:")
print(modified_text)


Модифицированный текст:
Wings An by written of Kalam is of Fire: Autobiography Abdul Dr. APJ Abdul Arun Kalam and Tiwari. This an of book is autobiography Dr. APJ president of former Abdul Kalam, India. This a journey boy story from a to the renowned of become young light on Rameswaram sheds scientist. It reflects will and living, work to dedication, strong led simple how hard success. It of cultural unity shows how impacts the lives also individuals. .


In [13]:
# Замена пассивных конструкций на активные
def activate_passive_sentences(text):
    active_text = re.sub(r'(\w+) was (\w+)', r'\2 \1', text)
    return active_text

# Добавление эмоциональных/оценочных элементов
def add_emotional_elements(text):
    emotional_words = [
        'весело', 'радостно', 'волнушкой', 'смеяться',
        'возбужденно', 'вдохновляюще', 'интригующе', 'взрывно',
        'трепетом', 'волнением', 'страстно', 'горячо', 'влюбленно'
    ]

    for word in emotional_words:
        text = re.sub(r'\bкроме того\b', f'{word} ', text)

    return text

# Изменение стиля изложения на более разговорный
def change_style_to_conversational(text):
    conversational_phrases = {
        'формальный': 'разговорный',
        'предложения': 'бабочки',
        'легче': 'просто',
        'воспринимаются': 'понимаются',
        'включение': 'добавление',
        'живым': 'активным',
        'выразительным': 'выражительным',
        'кроме того': 'еще',
        'также': 'а еще',
        'дополнительно': 'и так далее',
        'конечно': 'ну конечно',
        'действительно': 'правильно',
        'по сути': 'в общем',
        'в основном': 'в основном'
    }

    for key, value in conversational_phrases.items():
        text = text.replace(key, value)

    return text

# Пример исходного текста
original_text = "Предложения в активном залоге, как правило, легче воспринимаются. Кроме того, включение в текст слов, передающих эмоции или личное отношение автора, может сделать его более живым и выразительным."

# Применяем функции к тексту
activated_text = activate_passive_sentences(original_text)
emotional_text = add_emotional_elements(activated_text)
conversational_text = change_style_to_conversational(emotional_text)

print(conversational_text)

Предложения в активном залоге, как правило, просто понимаются. Кроме того, добавление в текст слов, передающих эмоции или личное отношение автора, может сделать его более активным и выражительным.


In [14]:
# Случайные перестановки букв в словах
def scramble_words_with_control(text, probability):
    words = re.findall(r'\b\w+\b', text)
    for i, word in enumerate(words):
        if random.random() < probability:
            scrambled_word = ''.join(random.sample(word, len(word)))
            text = text.replace(word, scrambled_word)
    return text

# Пример использования функции
probability=0.1
text_with_scramble = "Предложения в активном залоге, как правило, легче воспринимаются."
controlled_scrambled_text = scramble_words_with_control(text_with_scramble, probability=probability)

print(controlled_scrambled_text)

яПлдежинеро в активном залоге, как правило, легче воспринимаются.


In [21]:
# Замена на синонимы
def find_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return synonyms

def replace_synonyms(text):
    words = text.split()
    new_words = []
    for word in words:
        if word.lower() in wordnet.words():
            synonyms = find_synonyms(word)
            if len(synonyms) > 0:
                new_word = random.choice(synonyms)
                new_words.append(new_word)
            else:
                new_words.append(word)
        else:
            new_words.append(word)
    return ' '.join(new_words)

# Пример использования
text = "Wings of Fire: An Autobiography of Abdul Kalam is written by Dr. APJ Abdul Kalam and Arun Tiwari. This book is an autobiography of Dr. APJ Abdul Kalam"
print(replace_synonyms(text))

backstage of Fire: AN autobiography of Abdul Kalam is write by Dr. APJ Abdul Kalam and Arun Tiwari. This book is AN autobiography of doctor APJ Abdul Kalam


Pegasus + Paws + T5

In [ ]:
!pip install transformers sentencepiece >> None
!pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git >> None

from transformers import *

# models we gonna use for this tutorial
model_names = [
  "tuner007/pegasus_paraphrase",
  "Vamsi/T5_Paraphrase_Paws",
  "prithivida/parrot_paraphraser_on_T5", # Parrot
]

model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
tokenizer = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")

def get_paraphrased_sentences(model, tokenizer, sentence, num_return_sequences=5, num_beams=5):
  inputs = tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
  outputs = model.generate(
    **inputs,
    num_beams=num_beams,
    num_return_sequences=num_return_sequences,
  )
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

sentence = "Learning is the process of acquiring new understanding, knowledge, behaviors, skills, values, attitudes, and preferences."

get_paraphrased_sentences(model, tokenizer, sentence, num_beams=10, num_return_sequences=10)

get_paraphrased_sentences(model, tokenizer, "To paraphrase a source, you have to rewrite a passage without changing the meaning of the original text.", num_beams=10, num_return_sequences=10)

tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")

get_paraphrased_sentences(model, tokenizer, "paraphrase: " + "One of the best ways to learn is to teach what you've already learned")

from parrot import Parrot

parrot = Parrot()

phrases = [
  sentence,
  "One of the best ways to learn is to teach what you've already learned",
  "Paraphrasing is the process of coming up with someone else's ideas in your own words"
]

for phrase in phrases:
  print("-"*100)
  print("Input_phrase: ", phrase)
  print("-"*100)
  paraphrases = parrot.augment(input_phrase=phrase)
  if paraphrases:
    for paraphrase in paraphrases:
      print(paraphrase)

## 3. Метрика сравнения сгенеренного текста и исходного

In [ ]:
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Use CUDA if available, otherwise use CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

tokenizer = BertTokenizer.from_pretrained(model_name)

def compare_texts(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt').to(device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()
    return proba[0] # p(non-paraphrase), p(paraphrase)

print(compare_texts('Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода'))
print(compare_texts('Сегодня на улице хорошая погода', 'Отличная погодка сегодня выдалась'))